In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
import pandas as pd

# Read the XLSX file
df = pd.read_excel('CL-II-MisinformationData.xlsx')

# Save as CSV
df.to_csv('CL-II-MisinformationData.csv', index=False)


In [ ]:
# task1_prepare_dataset.py
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your CSV file
df = pd.read_csv('/content/CL-II-MisinformationData.csv')  # Replace with your CSV file name

# Split into train (80%) and temp (20%)
train_df, temp_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

# Split temp into validation (10%) and test (10%)
val_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=True, random_state=42)

# Save splits
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: '/content/CL-II-MisinformationData.csv'

In [ ]:
data = pd.read_csv('/content/train.csv')
data.head()

,tweet,label
0,#IndiaFightsCorona: India reports more than 90...,real
1,The continuing good news is that daily reporte...,real
2,• 3 people are currently in hospital one each ...,real
3,On the 15/03 NCDC directly contacted a Twitter...,real
4,India’s confirmed case count crosses ten lakh ...,fake


In [ ]:
import pandas as pd
import re

def preprocess_text(text):
  # Lowercase the text
  text = text.lower()
  # Remove Special Characters and Numbers
  text = re.sub(r'[^a-zA-Z\s]','', text)
  # Remove extra whitespace
  text = re.sub(r'\s+', ' ', text).strip()
  return text

# Load the trianing data
train_df = pd.read_csv('/content/train.csv')

# Apply preprocessing to the 'tweet' column
train_df['processed_tweet'] = train_df['tweet'].apply(preprocess_text)

In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
data['processed_tweet'] = data['tweet'].apply(preprocess_text) # Applying preprocessing on data DataFrame
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_tweet'])

# Create a DataFrame for the TF-IDF matrix
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenate the TF-IDF features with the original data
data = pd.concat([data, tfidf_df], axis=1)

# Display the first 5 rows with TF-IDF features
print(data.head())

                                               tweet label  \
0  #IndiaFightsCorona: India reports more than 90...  real   
1  The continuing good news is that daily reporte...  real   
2  • 3 people are currently in hospital one each ...  real   
3  On the 15/03 NCDC directly contacted a Twitter...  real   
4  India’s confirmed case count crosses ten lakh ...  fake   

                                     processed_tweet  aajtak  aamaadmiparty  \
0  indiafightscorona india reports more than reco...     0.0            0.0   
1  the continuing good news is that daily reporte...     0.0            0.0   
2  people are currently in hospital one each in a...     0.0            0.0   
3  on the ncdc directly contacted a twitter user ...     0.0            0.0   
4  indias confirmed case count crosses ten lakh m...     0.0            0.0   

   aamir  aampe  abandon  abbott  abia  ...  zealand  zealanders  zealands  \
0    0.0    0.0      0.0     0.0   0.0  ...      0.0         0.0       0.0

In [ ]:
# Prepare data for KNN
X = tfidf_matrix
y = data['label'].map(lambda x: 0 if x == 'fake' else 1)  # Assuming 'label' column contains 'fake' or 'real'

# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.2, random_state=42)

# KNN Model Training
knn_classifier = KNeighborsClassifier(n_neighbors=5)  # You can adjust the number of neighbors
knn_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90       899
           1       1.00      1.00      1.00      1696

   micro avg       0.97      0.96      0.97      2595
   macro avg       0.96      0.95      0.95      2595
weighted avg       0.97      0.96      0.97      2595
 samples avg       0.98      0.97      0.97      2595



In [ ]:
# Hyperparameter tuning for KNN using GridSearchCV
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'cosine']
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='f1_macro') # Use 5-fold cross-validation
grid_search.fit(X_train, y_train)

print("Best hyperparameters:", grid_search.best_params_)
print("Best F1 score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_knn_classifier = grid_search.best_estimator_
y_pred = best_knn_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
# Prepare data for Logistic Regression
X = tfidf_matrix
y = train_df['label'].map(lambda x: 0 if x == 'fake' else 1)  # Assuming 'label' column contains 'fake' or 'real'

# Convert y to a 1D NumPy array to avoid the ValueError
y = y.values.ravel()

# Split data into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression Model Training
logistic_regression_classifier = LogisticRegression(max_iter=1000) # Increased max_iter for converegence
logistic_regression_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = logistic_regression_classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

In [ ]:
# LinearSVC Model Training
linear_svc_classifier = LinearSVC()
linear_svc_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = linear_svc_classifier.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


In [ ]:
# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(data['processed_tweet'])

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# KMeans Clustering
kmeans = KMeans(n_clusters=2, random_state=42)  # Assuming 2 clusters for fake and real
kmeans.fit(tfidf_matrix)

# Get Cluster labels for each data point
cluster_labels = kmeans.labels_

# Add cluster labels to the DataFrame
data['cluster'] = cluster_labels

# Display the first 5 rows with cluster labels
print(data.head())

                                               tweet label  \
0  #IndiaFightsCorona: India reports more than 90...  real   
1  The continuing good news is that daily reporte...  real   
2  • 3 people are currently in hospital one each ...  real   
3  On the 15/03 NCDC directly contacted a Twitter...  real   
4  India’s confirmed case count crosses ten lakh ...  fake   

                                     processed_tweet  aajtak  aamaadmiparty  \
0  indiafightscorona india reports more than reco...     0.0            0.0   
1  the continuing good news is that daily reporte...     0.0            0.0   
2  people are currently in hospital one each in a...     0.0            0.0   
3  on the ncdc directly contacted a twitter user ...     0.0            0.0   
4  indias confirmed case count crosses ten lakh m...     0.0            0.0   

   aamir  aampe  abandon  abbott  abia  ...  zealand  zealanders  zealands  \
0    0.0    0.0      0.0     0.0   0.0  ...      0.0         0.0       0.0

In [ ]:
y_pred_kmeans = data['cluster']
print("KMeans Classification Report:")
print(classification_report(y, y_pred_kmeans))

KMeans Classification Report:
              precision    recall  f1-score   support

           0       0.03      0.01      0.02      4029
           1       0.45      0.72      0.55      4451

    accuracy                           0.38      8480
   macro avg       0.24      0.37      0.28      8480
weighted avg       0.25      0.38      0.30      8480



In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Neural Network Model Training
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
mlp_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_mlp = mlp_classifier.predict(X_test)

# Evaluate the model
print("Neural Network Classification Report:")
print(classification_report(y_test, y_pred_mlp))


Neural Network Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.92      0.91       797
           1       0.93      0.92      0.92       899

    accuracy                           0.92      1696
   macro avg       0.92      0.92      0.92      1696
weighted avg       0.92      0.92      0.92      1696



In [ ]:
# Gradient Boosting Model Training
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_classifier.predict(X_test)

# Evalauate the model
print("Gradient Boosting Classification Report")
print(classification_report(y_test, y_pred_gb))

Implementation of pytorch


In [ ]:
import torch
import torch.nn as nn

# Define activation functions
def sigmoid(x):
  return 1 / (1 + torch.exp(-x))

def tanh(x):
  return torch.tanh(x)

def relu(x):
  return torch.relu(x)

def leaky_relu(x, negative_slope=0.01):
  return torch.nn.functional.leaky_relu(x, negative_slope=negative_slope)

# Example usage:
x = torch.tensor([-1.0, 0.0, 1.0])

print("Sigmoid:", sigmoid(x))
print("Tanh:", tanh(x))
print("ReLU:", relu(x))
print("Leaky ReLU:", leaky_relu(x))


# Using PyTorch's built-in activation functions (more efficient)
sigmoid_activation = nn.Sigmoid()
tanh_activation = nn.Tanh()
relu_activation = nn.ReLU()
leaky_relu_activation = nn.LeakyReLU(negative_slope=0.01)

print("\nUsing PyTorch built-in functions:")
print("Sigmoid:", sigmoid_activation(x))
print("Tanh:", tanh_activation(x))
print("ReLU:", relu_activation(x))
print("Leaky ReLU:", leaky_relu_activation(x))


In [ ]:
# Define the cost function (loss function)
def cost_function(y_pred, y_true):
    mse_loss = nn.MSELoss()  # Use PyTorch's built-in MSE loss function
    return mse_loss(y_pred, y_true)

# Example usage:
y_predicted = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y_actual = torch.tensor([1.5, 1.8, 3.2])
loss = cost_function(y_predicted, y_actual)

print("MSE loss:", loss)


# Example using binary cross-entropy loss
def binary_cross_entropy_cost(y_pred, y_true):
    bce_loss = nn.BCELoss()
    return bce_loss(y_pred, y_true)

# Example usage (make sure y_pred is between 0 and 1, e.g., use a sigmoid activation)
y_predicted_prob = torch.sigmoid(torch.tensor([1.0, 2.0, 3.0], requires_grad=True))  # Example: Using sigmoid to get probabilities
y_actual_binary = torch.tensor([1.0, 0.0, 1.0]) #Example binary target labels
loss = binary_cross_entropy_cost(y_predicted_prob, y_actual_binary)
print("Binary Cross-Entropy loss:", loss)


In [ ]:

data['new_attribute'] = 0
print(data.head())


In [ ]:
import torch

# Assuming 'data' DataFrame and 'tfidf_matrix' are already defined as in your code.

# Convert the TF-IDF matrix to a PyTorch tensor
X_tensor = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)

# Convert the labels to a PyTorch tensor
# Instead of y.values, directly use the label column from the DataFrame.
y_tensor = torch.tensor(data['label'].map(lambda x: 0 if x == 'fake' else 1).values, dtype=torch.long) # Use torch.long for classification labels

# Print shapes to verify
print("X_tensor shape:", X_tensor.shape)
print("y_tensor shape:", y_tensor.shape)

In [ ]:
import torch

# Assuming 'data' DataFrame and 'tfidf_matrix' are already defined as in your code.

# Convert the TF-IDF matrix to a PyTorch tensor
X_tensor = torch.tensor(tfidf_matrix.toarray(), dtype=torch.float32)

# Convert the labels to a PyTorch tensor
# Instead of y.values, directly use the label column from the DataFrame.
y_tensor = torch.tensor(data['label'].map(lambda x: 0 if x == 'fake' else 1).values, dtype=torch.long) # Use torch.long for classification labels

# Print shapes to verify
print("X_tensor shape:", X_tensor.shape)
print("y_tensor shape:", y_tensor.shape)


In [ ]:
# Add dropout regularization to the model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate) # Dropout layer
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x) # Apply dropout
        x = self.fc2(x)
        return


In [ ]:
# Define the neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
input_size = X_tensor.shape[1]  # Number of features
hidden_size = 64  # Number of neurons in the hidden layer
output_size = 2  # Number of output classes (fake/real)
learning_rate = 0.001
num_epochs = 10

# Initialize the model, loss function, and optimizer
model = NeuralNetwork(input_size, hidden_size, output_size)
# Changed the criterion to BCELoss, since output is not one-hot encoded
criterion = nn.BCELoss()  # Use cross-entropy loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_tensor)
    # Apply sigmoid for probabilities
    outputs = torch.sigmoid(outputs)
    loss = criterion(outputs, y_tensor.float()) # The target should be class probabilities (FloatTensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions on the entire dataset (for demonstration)
with torch.no_grad():  # No need to track gradients during prediction
    predicted_probabilities = torch.sigmoid(model(X_tensor))  # Get probabilities
    predicted_labels = (predicted_probabilities >= 0.5).int()  # Get predicted class labels (0 or 1)

# Print the classification report
print(classification_report(y_tensor.cpu(), predicted_labels.cpu())) # Move tensors to CPU for sklearn

In [ ]:
class RNN2DFlatten(nn.Module):
    def __init__(self, input_height, input_width, hidden_size, num_layers, num_classes):
        super(RNN2DFlatten, self).__init__()
        self.input_height = input_height
        self.input_width = input_width
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Flatten 2D input to 1D (height * width)
        self.rnn = nn.RNN(input_height * input_width, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x shape: (batch_size, sequence_length, height, width)
        batch_size = x.size(0)
        seq_length = x.size(1)

        # Flatten spatial dimensions
        x = x.view(batch_size, seq_length, -1)

        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output[:, -1, :])  # Take last time step's output
        return output, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size)

# Example usage for text classification
class TextDataset(Dataset):
    def __init__(self, texts, labels, char_to_idx, max_length):
        self.texts = [self.text_to_tensor(text, char_to_idx, max_length) for text in texts]
        self.labels = labels

    def text_to_tensor(self, text, char_to_idx, max_length):
        indices = [char_to_idx[char] for char in text[:max_length]]
        if len(indices) < max_length:
            indices += [0] * (max_length - len(indices))  # Padding
        return torch.tensor(indices, dtype=torch.long)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        hidden = model.init_hidden(inputs.size(0)).to(device)

        optimizer.zero_grad()
        outputs, _ = model(inputs, hidden)
        loss = criterion(outputs, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # Gradient clipping
        optimizer.step()


In [ ]:

import torch.nn.functional as F

def generate(model, start_seq, char_to_idx, idx_to_char, length=100, temperature=1.0, device='cpu'):
    model.eval()
    input_seq = torch.tensor([char_to_idx[c] for c in start_seq], dtype=torch.long).unsqueeze(0).to(device)
    hidden = model.init_hidden(1).to(device)

    generated_seq = start_seq

    with torch.no_grad():
        for i in range(len(start_seq) - 1):
            _, hidden = model(input_seq[:, i].unsqueeze(1), hidden)

        last_char = input_seq[:, -1].unsqueeze(1)

        for _ in range(length):
            output, hidden = model(last_char, hidden)  # output shape: (1, output_size)
            output_dist = output.data.view(-1).div(temperature).exp()
            top_char = torch.multinomial(output_dist, 1)[0]

            predicted_char = idx_to_char[top_char.item()]
            generated_seq += predicted_char

            last_char = torch.tensor([[top_char]], dtype=torch.long).to(device)

    return generated_seq


In [ ]:

import torch.nn as nn

class LSTM2D(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM2D, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out


In [ ]:
# Example parameters
batch_size = 4
seq_length = 10
input_size = 8
hidden_size = 16
num_layers = 2
num_classes = 3

# Create model
model = LSTM2D(input_size, hidden_size, num_layers, num_classes)

# Dummy input: batch of sequences
x = torch.randn(batch_size, seq_length, input_size)

# Forward pass
output = model(x)

print("Output shape:", output.shape)
print("Output tensor:", output)
